In [2]:
from kubernetes import client, config, watch
import pandas as pd
import matplotlib.pyplot as plt

from prometheus_api_client import PrometheusConnect

# from datetime import datetime, timedelta
import time
from difflib import diff_bytes
import matplotlib.pyplot as plt
import kubernetes.client.models.v1_pod as v1pod

In [4]:
# test the connecting with Prometheus
# Kubernetes Config
config.load_kube_config()
v1 = client.CoreV1Api()

# Prometheus Config
#prom_url = "http://<PROMETHEUS_SERVER_IP>:<PORT>"
# prom_url = "http://10.110.188.57:9090"
prom_url = "http://prometheus-stack-kube-prom-prometheus.monitoring.svc.cluster.local:9090"
prom = PrometheusConnect(url=prom_url, disable_ssl=True)
#test prom connection
response = prom.custom_query(query="instance:node_network_receive_bytes:rate:sum")
print(response)

[{'metric': {'__name__': 'instance:node_network_receive_bytes:rate:sum', 'instance': '172.26.128.228:9100'}, 'value': [1709723262.259, '141242.7']}, {'metric': {'__name__': 'instance:node_network_receive_bytes:rate:sum', 'instance': '172.26.128.30:9100'}, 'value': [1709723262.259, '4273.426666666666']}, {'metric': {'__name__': 'instance:node_network_receive_bytes:rate:sum', 'instance': '172.26.130.22:9100'}, 'value': [1709723262.259, '82282.89333333333']}, {'metric': {'__name__': 'instance:node_network_receive_bytes:rate:sum', 'instance': '172.26.130.82:9100'}, 'value': [1709723262.259, '4657.173333333333']}, {'metric': {'__name__': 'instance:node_network_receive_bytes:rate:sum', 'instance': '172.26.132.142:9100'}, 'value': [1709723262.259, '13537.453333333333']}, {'metric': {'__name__': 'instance:node_network_receive_bytes:rate:sum', 'instance': '172.26.132.241:9100'}, 'value': [1709723262.259, '5554.799999999999']}, {'metric': {'__name__': 'instance:node_network_receive_bytes:rate:su

In [8]:
# find all nodes and its ip in the cluster, store the nodename and ip in a dictionary
node_ip = {}
nodes = v1.list_node()
for node in nodes.items:
    for address in node.status.addresses:
        if address.type == "InternalIP":
            node_ip[node.metadata.name] = address.address
print(node_ip)

{'k8s-master': '172.26.128.228', 'k8s-worker-1': '172.26.128.30', 'k8s-worker-2': '172.26.132.91', 'k8s-worker-3': '172.26.133.31', 'k8s-worker-4': '172.26.132.241', 'k8s-worker-5': '172.26.132.142', 'k8s-worker-6': '172.26.133.55', 'k8s-worker-7': '172.26.130.22', 'k8s-worker-8': '172.26.130.82', 'k8s-worker-9': '172.26.133.118'}


In [9]:
import requests

# find all nodes and its ip in the cluster, store the nodename and ip in a dictionary
node_ip = {}
nodes = v1.list_node()
for node in nodes.items:
    for address in node.status.addresses:
        if address.type == "InternalIP":
            node_ip[node.metadata.name] = address.address
print(node_ip)

# URL of your Prometheus server
PROMETHEUS = "http://prometheus-stack-kube-prom-prometheus.monitoring.svc.cluster.local:9090"

# Your Prometheus query
query = 'instance:node_network_receive_bytes:rate:sum'
# query = 'sum(rate(node_network_receive_bytes_total[1m])) * 60'


# Construct the full URL for the query
url = f'{PROMETHEUS}/api/v1/query'

# Parameters for the query
params = {
    'query': query,
    # Optionally, you can specify the exact time you want to query for
    # 'time': '2024-02-27T12:00:00Z',
}


def find_loweset_bandwidth():
     # Make the HTTP GET request
    response = requests.get(url, params=params)

    node_bandwidth = {}
    # Check if the request was successful
    if response.status_code == 200:
        result = response.json()
        # Print the entire response
        # print(result)
        # Extract and print just the result data
        data = result.get('data', {}).get('result', [])
        if data:
            for item in data:
                # Assuming the query returns vector results, iterate and print
                metric_name = item.get('metric', {}).get('__name__', 'Unknown metric')
                instance_name = item.get('metric', {}).get('instance', 'Unknown instance')
                value = item.get('value', [])[1]  # The value is a [timestamp, value] pair
                # print(f'node {instance_name} has network bandith with :{value}')
                
                # save the instance name and value to a dictionary
                instance_ip = instance_name.split(":")[0] # get the ip of the instance, Eg: change this string "172.26.128.228:9100'" into "172.26.128.228"
                node_bandwidth[instance_ip] = float(value) # change string value into float
        else:
            print("No data returned")
    else:
        print(f"Failed to retrieve data: {response.status_code} - {response.text}")
    print(node_bandwidth)
    
    
    
    lowest_bandwidth = node_bandwidth[list(node_bandwidth.keys())[0]]
    # print(lowest_bandwidth)
    lowest_bandwidth_node = ""
    for node, bandwidth in node_bandwidth.items():
        if bandwidth < lowest_bandwidth:
            lowest_bandwidth = bandwidth
            lowest_bandwidth_node = node
            
    return lowest_bandwidth_node, lowest_bandwidth

#find the node name with lowest bandwidth
# lowest_bandwidth_node, lowest_bandwidth = find_loweset_bandwidth(node_bandwidth)
# print(f"the node with lowest bandwidth is {lowest_bandwidth_node} with bandwidth {lowest_bandwidth}")



In [6]:
# create  a pod on a given node and namespace
def create_pod(name, namespace, image, cpu_request, memory_request, node_name):
    # Configure the Kubernetes client
    config.load_kube_config()

    # Create API clients
    v1 = client.CoreV1Api()

    # Define the Pod spec
    pod = v1pod.V1Pod()
    # specify the nodename for the pod
    
    pod.metadata = client.V1ObjectMeta(name=name, namespace=namespace)
    pod.spec = client.V1PodSpec(containers=[client.V1Container(name=name, image=image, resources=client.V1ResourceRequirements(
        requests={"cpu": cpu_request, "memory": memory_request}))], node_name=node_name)

    # Create the Pod
    v1.create_namespaced_pod(namespace=namespace, body=pod)

    print(f"Created Pod {name} in namespace {namespace} with image {image} and resource requests: CPU={cpu_request}, Memory={memory_request} on node {node_name}")



In [11]:
# create pods with different names with loop for testing, eaching creation pod will take 60 seconds
for i in range(1,10,1):
    a, b =find_loweset_bandwidth()
    print(f"the node with lowest bandwidth is {a} with bandwidth {b}")
    # find the nodename based on the ip
    for node, ip in node_ip.items():
        if ip == a:
            pod_binding_node = node
    
    create_pod(name=f"test-pod-{i}", namespace="nginx-app-test", image="nginx", cpu_request="100m", memory_request="128Mi", node_name=pod_binding_node)
    time.sleep(60)

{'172.26.128.228': 146217.84, '172.26.128.30': 4002.973333333333, '172.26.130.22': 81927.04666666665, '172.26.130.82': 5248.139999999999, '172.26.132.142': 13618.78, '172.26.132.241': 5502.466666666667, '172.26.132.91': 6840.44, '172.26.133.118': 7263.873333333333, '172.26.133.31': 5801.88, '172.26.133.55': 4056.9933333333333}
the node with lowest bandwidth is 172.26.128.30 with bandwidth 4002.973333333333
Created Pod test-pod-1 in namespace nginx-app-test with image nginx and resource requests: CPU=100m, Memory=128Mi on node k8s-worker-1
{'172.26.128.228': 146724.74, '172.26.128.30': 6885.033333333332, '172.26.130.22': 82682.95999999999, '172.26.130.82': 7169.473333333333, '172.26.132.142': 13758.2, '172.26.132.241': 5569.099999999999, '172.26.132.91': 7488.719999999999, '172.26.133.118': 7825.179999999999, '172.26.133.31': 5319.113333333333, '172.26.133.55': 5631.54}
the node with lowest bandwidth is 172.26.133.31 with bandwidth 5319.113333333333
Created Pod test-pod-2 in namespace n

In [ ]:
from time import sleep


while(True):
    sleep(3) # sleep for 5 seconds
    a, b =find_loweset_bandwidth()
    print(f"the node with lowest bandwidth is {a} with bandwidth {b}")
    # find the nodename based on the ip
    for node, ip in node_ip.items():
        if ip == a:
            pod_binding_node = node
    print(f"the node with lowest bandwidth is {pod_binding_node} with bandwidth {b}")

    
    

In [ ]:
from os import getenv
from json import loads as json_loads
import random

from kubernetes import config, watch
from kubernetes.client import ApiClient, CoreV1Api, V1ObjectReference, V1ObjectMeta, V1Binding, Configuration
from kubernetes.client.rest import ApiException, RESTClientObject

from logging import basicConfig, getLogger, INFO

formatter = " %(asctime)s | %(levelname)-6s | %(process)d | %(threadName)-12s |" \
            " %(thread)-15d | %(name)-30s | %(filename)s:%(lineno)d | %(message)s |"
basicConfig(level=INFO, format=formatter)
logger = getLogger("meetup-scheduler")

V1_CLIENT = None  # type: CoreV1Api
SCHEDULE_STRATEGY = "schedulingStrategy=meetup"
_NOSCHEDULE_TAINT = "NoSchedule"


def _get_ready_nodes(v1_client, filtered=True):
    ready_nodes = []
    try:
        for n in v1_client.list_node().items:
            if n.metadata.labels.get("noCustomScheduler") == "yes":
                logger.info(f"Skipping Node {n.metadata.name} since it has noCustomScheduler label")
                continue
            if filtered:
                if not n.spec.unschedulable:
                    no_schedule_taint = False
                    if n.spec.taints:
                        for taint in n.spec.taints:
                            if _NOSCHEDULE_TAINT == taint.to_dict().get("effect", None):
                                no_schedule_taint = True
                                break
                    if not no_schedule_taint:
                        for status in n.status.conditions:
                            if status.status == "True" and status.type == "Ready" and n.metadata.name:
                                ready_nodes.append(n.metadata.name)
                    else:
                        logger.error("NoSchedule taint effect on node %s", n.metadata.name)
                else:
                    logger.error("Scheduling disabled on %s ", n.metadata.name)
            else:
                if n.metadata.name:
                    ready_nodes.append(n.metadata.name)
        logger.info("Nodes : %s, Filtered: %s", ready_nodes, filtered)
    except ApiException as e:
        logger.error(json_loads(e.body)["message"])
        ready_nodes = []
    return ready_nodes

# later add the funtcion for choosing the node from schedulable nodes


def _get_schedulable_node(v1_client):
    node_list = _get_ready_nodes(v1_client)
    if not node_list:
        return None
    available_nodes = list(set(node_list))
    return random.choice(available_nodes) # later add the logic for choosing the node from the list of available nodes


def schedule_pod(v1_client, name, node, namespace="default"):
    target = V1ObjectReference()
    target.kind = "Node"
    target.apiVersion = "v1"
    target.name = node
    meta = V1ObjectMeta()
    meta.name = name
    body = V1Binding(api_version=None, kind=None, metadata=meta, target=target)
    logger.info("Binding Pod: %s  to  Node: %s", name, node)
    return v1_client.create_namespaced_pod_binding(name, namespace, body)

def watch_pod_events():
    V1_CLIENT = CoreV1Api()
    while True:
        try:
            logger.info("Checking for pod events....")
            try:
                watcher = watch.Watch()
                for event in watcher.stream(V1_CLIENT.list_pod_for_all_namespaces, label_selector=SCHEDULE_STRATEGY, timeout_seconds=20):
                    logger.info(f"Event: {event['type']} {event['object'].kind}, {event['object'].metadata.namespace}, {event['object'].metadata.name}, {event['object'].status.phase}")
                    if event["object"].status.phase == "Pending":
                        try:
                            logger.info(f'{event["object"].metadata.name} needs scheduling...')
                            pod_namespace = event["object"].metadata.namespace
                            pod_name = event["object"].metadata.name
                            service_name = event["object"].metadata.labels["serviceName"]
                            logger.info("Processing for Pod: %s/%s", pod_namespace, pod_name)
                            node_name = _get_schedulable_node(V1_CLIENT)
                            if node_name:
                                logger.info("Namespace %s, PodName %s , Node Name: %s  Service Name: %s",
                                            pod_namespace, pod_name, node_name, service_name)
                                res = schedule_pod(V1_CLIENT, pod_name, node_name, pod_namespace)
                                logger.info("Response %s ", res)
                            else:
                                logger.error(f"Found no valid node to schedule {pod_name} in {pod_namespace}")
                        except ApiException as e:
                            logger.error(json_loads(e.body)["message"])
                        except ValueError as e:
                            logger.error("Value Error %s", e)
                        except:
                            logger.exception("Ignoring Exception")
                logger.info("Resetting k8s watcher...")
            except:
                logger.exception("Ignoring Exception")
            finally:
                del watcher
        except:
            logger.exception("Ignoring Exception & listening for pod events")

def main():
    logger.info("Initializing the meetup scheduler...")
    logger.info("Watching for pod events...")
    watch_pod_events()


if __name__ == "__main__":
    config.load_incluster_config()
    main()